In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Use python 3.8.10 to run this notebook !

### Install following packages  

In [6]:
# !pip install tqdm==4.61.2
# !pip install numpy==1.20.3
# !pip install scikit-learn==0.24.2
# !pip install pandas==1.4.1
# !pip install umap-learn==0.4.6
# !pip install anndata==0.7.4
# !pip install ripser==0.6.1
# !pip install Cython==0.29.21
# !pip install numba==0.51.2
# !pip install seaborn
# !pip install tables==3.6.1
# !pip install legacy-api-wrap==1.2
# !pip install scikit-network==0.24.0

In [9]:
import matplotlib.pyplot as plt
from anndata import AnnData
import scanpy_modified as scanpy
from tqdm import tqdm
import numpy as np
from numpy import inf
from ripser import Rips
from scipy.spatial import distance
from numpy import linalg as LA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import sklearn
import pandas as pd
from sklearn.manifold import TSNE
import umap
import glob, os
from sklearn.decomposition import PCA
import sys
import time
import random
import seaborn as sns
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from utils import features_dpt_entropy, features_homology_dpt_entropy, features_vector
from utils import features_ripley_dpt_v2, preprocessing, features_avg_connection_dpt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sknetwork.data import karate_club, painters, movie_actor
from sknetwork.clustering import Louvain, modularity, bimodularity
from sklearn.neighbors import kneighbors_graph
from matplotlib import pyplot as plt
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
import simdata_generator as dataset

eps = sys.float_info.epsilon
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Generating Simulated Datasets

In [19]:
# number of datasets for each type of simulated dataset
repeat = 3000

In [20]:
df_cluster = []
for i in tqdm(range(repeat),position=0, leave=True):
    sim = dataset.gen_cluster(num=np.random.randint(500,1500))
    df_cluster.append(sim)

df_traj = []
for i in tqdm(range(repeat),position=0, leave=True):
    sim = dataset.gen_trajectory(num=np.random.randint(500,1500))
    df_traj.append(sim)
  

100%|████████████████████████████████████| 3000/3000 [00:00<00:00, 12860.14it/s]


In [21]:
df_rand_cluster = []
for i in tqdm(range(repeat),position=0, leave=True):
    sim, ind = dataset.gen_random(num=np.random.randint(500,1500),mode='cluster')
    df_rand_cluster.append(sim)

df_rand_traj = []
for i in tqdm(range(repeat),position=0, leave=True):
    sim, ind = dataset.gen_random(num=np.random.randint(500,1500),mode='trajectory')
    df_rand_traj.append(sim)
    

100%|█████████████████████████████████████| 3000/3000 [00:00<00:00, 7658.65it/s]


In [9]:
all_df = df_cluster + df_rand + df_rand_cluster + df_rand_traj

In [92]:
# if you want to save scatter plots of each dataset, run this cell! 
ct=0
for a in tqdm(all_df):
    ct += 1
    fig = plt.figure(figsize=(1.5,1.5))
    plt.scatter(a[:,0],a[:,1],s=1)
    fig.patch.set_facecolor('white')
    fig.patch.set_edgecolor('black')  
    fig.patch.set_linewidth('1') 
    plt.axis('off')
    fig.savefig('scatterplots/{}.png'.format(ct))
    plt.close()

100%|████████████████████████████████████████████████████████████████████████████| 18000/18000 [05:34<00:00, 53.84it/s]


In [16]:
# generation of scores
META_SCORES = []
ct = 0
for df in tqdm(all_df, position=0, leave=True):
    df = preprocessing(df)
    sc1 = features_dpt_entropy(df, num_bins = 10)
    sc2 = features_homology_dpt_entropy(df,num_bins = 3)
    sc3 = features_vector(df)
    sc4 = features_ripley_dpt_v2(df)
    sc5 = features_avg_connection_dpt(df)
    scores = [sc1,sc2,sc3,sc4,sc5]
    
    META_SCORES.append(scores)
    if ct % 3000 == 0:
        np.save('tmp_{}.npy'.format(ct), np.array(META_SCORES))
    ct += 1 

100%|███████████████████████████████████████████████████████████████████████████| 9000/9000 [33:25:16<00:00, 13.37s/it]


In [18]:
# save the meatascore
np.save('metascore_12000.npy', np.array(META_SCORES))